<a href="https://colab.research.google.com/github/IntelAntique/Cotiviti/blob/main/Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
df = pd.read_csv('Diabetes-Data.csv', index_col = False)
df = df.drop(columns="Id")
n = len(df.columns)
dataset = df.values

train, test = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [ ]:
training_data = torch.tensor(train)
test_data = torch.tensor(test)

print(training_data[:, n - 1: n])
print(training_data[:, 0: n - 1])

tensor([[0.],
        [1.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]], dtype=torch.float64)
tensor([[2.0000e+00, 1.1200e+02, 7.8000e+01,  ..., 3.9400e+01, 1.7500e-01,
         2.4000e+01],
        [5.0000e+00, 1.5800e+02, 8.4000e+01,  ..., 3.9400e+01, 3.9500e-01,
         2.9000e+01],
        [1.0000e+00, 1.0100e+02, 5.0000e+01,  ..., 2.4200e+01, 5.2600e-01,
         2.6000e+01],
        ...,
        [7.0000e+00, 1.9600e+02, 9.0000e+01,  ..., 3.9800e+01, 4.5100e-01,
         4.1000e+01],
        [3.0000e+00, 1.1200e+02, 7.4000e+01,  ..., 3.1600e+01, 1.9700e-01,
         2.5000e+01],
        [1.0000e+00, 1.3300e+02, 1.0200e+02,  ..., 3.2800e+01, 2.3400e-01,
         4.5000e+01]], dtype=torch.float64)


In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for data in test_dataloader:
    X, y = data[:, : n - 1], data[:, n - 1]
    print(f"Shape of X [N, C]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C]: torch.Size([64, 8])
Shape of y: torch.Size([64]) torch.float64


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [ ]:
input = 8
output = 2

class BinaryClassification(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Linear(input, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output)
        )

    def forward(self, x):
        return self.sequential(x)

model = BinaryClassification(input, output).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader):
        X, y = data[:, : n - 1].type(torch.float).to(device), data[:, n - 1].type(torch.long).to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data in dataloader:
            X, y = data[:, : n - 1].type(torch.float).to(device), data[:, n - 1].type(torch.long).to(device)
            pred = model(X)
            print(pred)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Streaming output truncated to the last 5000 lines.
loss: 0.047460  [ 1984/ 2215]
Test Error: 
 Accuracy: 96.6%, Avg loss: 0.166977 

Epoch 446
-------------------------------
loss: 0.035786  [   64/ 2215]
loss: 0.030118  [  704/ 2215]
loss: 0.022253  [ 1344/ 2215]
loss: 0.069950  [ 1984/ 2215]
Test Error: 
 Accuracy: 96.7%, Avg loss: 0.164000 

Epoch 447
-------------------------------
loss: 0.036311  [   64/ 2215]
loss: 0.029036  [  704/ 2215]
loss: 0.022743  [ 1344/ 2215]
loss: 0.029410  [ 1984/ 2215]
Test Error: 
 Accuracy: 96.4%, Avg loss: 0.162739 

Epoch 448
-------------------------------
loss: 0.026490  [   64/ 2215]
loss: 0.026585  [  704/ 2215]
loss: 0.026299  [ 1344/ 2215]
loss: 0.066915  [ 1984/ 2215]
Test Error: 
 Accuracy: 96.9%, Avg loss: 0.163795 

Epoch 449
-------------------------------
loss: 0.039101  [   64/ 2215]
loss: 0.018963  [  704/ 2215]
loss: 0.024680  [ 1344/ 2215]
loss: 0.040907  [ 1984/ 2215]
Test Error: 
 Accuracy: 96.2%, Avg loss: 0.175979 

Epoch 450
-